# EXPLORATION 15. 번역기를 만들어보자

## Import

In [1]:
import os
import pandas as pd
import numpy as np
import re

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

## Data

In [2]:
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 178009


,eng,fra,cc
131022,I didn't really feel like going out.,Je n'avais pas vraiment envie de sortir.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
72046,Soak this sponge in water.,Fais tremper cette éponge dans l'eau.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
92046,I'll see if I can find a cab.,Je vais voir si je peux trouver un taxi.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
96564,Don't think about it too much.,N'y pensez pas trop.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
146247,"I told you not to call me Tom, didn't I?","Je t'ai dit de ne pas m'appeler Tom, non ?",CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [3]:
lines = lines[['eng', 'fra']][:33000] # 5만개 샘플 사용
lines.sample(5)

,eng,fra
17203,We're here alone.,Nous sommes ici seules.
17364,What should I do?,Que devrais-je faire ?
26960,Tom works too much.,Thomas travaille trop.
27783,You intimidate Tom.,Vous intimidez Tom.
7518,Don't judge me.,Ne me jugez pas.


## Preprocessing

### preprocess sentence

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
  
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!]+", " ", sentence)
    
    sentence = sentence.strip()
    sentence = sentence.split(" ")
    
    return sentence

In [5]:
def preprocess_sentence_decoder(sentence):
    sentence = sentence.lower().strip()
  
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!]+", " ", sentence)
    
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    sentence = sentence.split(" ")
    
    return sentence

In [6]:
lines.eng = lines.eng.apply(lambda x : preprocess_sentence(x))
lines.fra = lines.fra.apply(lambda x : preprocess_sentence_decoder(x))

In [7]:
lines.eng.sample(5)

8206          [i, need, to, know, .]
28611    [does, it, even, matter, ?]
26701     [tom, began, to, laugh, .]
9691           [were, you, happy, ?]
2559              [i, m, talking, .]
Name: eng, dtype: object

In [8]:
lines.fra.sample(5)

29974    [<start>, il, faut, que, j, y, aille, maintena...
25249    [<start>, le, chien, est, il, encha, n, ?, <end>]
11430             [<start>, j, tais, horrifi, e, ., <end>]
10299          [<start>, avez, vous, fait, ceci, ?, <end>]
19820           [<start>, j, en, choisirai, une, ., <end>]
Name: fra, dtype: object

### Vocabulary

In [9]:
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(lines.eng)

In [10]:
print(eng_tokenizer.word_index)

{'.': 1, 'i': 2, 'you': 3, '?': 4, 'it': 5, 'a': 6, 's': 7, 'is': 8, 're': 9, 'we': 10, 'tom': 11, 't': 12, 'm': 13, 'the': 14, 'he': 15, 'that': 16, 'to': 17, 'this': 18, 'me': 19, 'are': 20, 'was': 21, 'do': 22, 'can': 23, 'they': 24, 'my': 25, 'have': 26, 'your': 27, 'not': 28, 'don': 29, 'go': 30, 'she': 31, '!': 32, 'what': 33, 'no': 34, 'll': 35, 'be': 36, 'did': 37, 'all': 38, 'like': 39, 'in': 40, 'here': 41, 'very': 42, 'let': 43, 'how': 44, 'on': 45, 'one': 46, 'up': 47, 'want': 48, 'need': 49, 'him': 50, 'of': 51, 'get': 52, 'there': 53, 'know': 54, 'love': 55, 'now': 56, 'just': 57, 'out': 58, 'so': 59, 'too': 60, 'who': 61, 'for': 62, 'come': 63, 'were': 64, 'good': 65, 'us': 66, 'got': 67, 've': 68, 'at': 69, 'has': 70, 'please': 71, 'look': 72, 'help': 73, 'stop': 74, 'see': 75, 'am': 76, 'will': 77, 'where': 78, 'had': 79, 'take': 80, 'her': 81, 'his': 82, 'why': 83, 'give': 84, 'didn': 85, 'feel': 86, 'must': 87, 'home': 88, 'back': 89, 'happy': 90, 'car': 91, 'made': 

In [11]:
input_text = eng_tokenizer.texts_to_sequences(lines.eng)
input_text[:3]

[[30, 1], [1132, 1], [1132, 1]]

In [12]:
fra_tokenizer = Tokenizer()
fra_tokenizer.fit_on_texts(lines.fra)
target_text = fra_tokenizer.texts_to_sequences(lines.fra)

In [13]:
target_text[:3]

[[1, 91, 12, 2], [1, 1068, 12, 2], [1, 1068, 3, 2]]

In [14]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 4662
프랑스어 단어장의 크기 : 7326


In [15]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 17


In [16]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4662
프랑스어 단어장의 크기 : 7326
영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 17


### for teacher forcing

In [17]:
sos_token = '<start>'
eos_token = '<end>'

encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

### padding

In [19]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 17)
프랑스어 출력데이터의 크기(shape) : (33000, 17)


### train validation split 

In [20]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [21]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(30000, 8)
(30000, 17)
(30000, 17)
(3000, 8)
(3000, 17)
(3000, 17)


## Train

### encoder

In [22]:
encoder_inputs = Input(shape=(None,))
# encoder embedding
enc_emb = Embedding(eng_vocab_size, 256, input_length=max_eng_seq_len)(encoder_inputs)
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(units = 256, return_state = True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

### decoder

In [23]:
decoder_inputs = Input(shape=(None,))
# decoder embedding
dec_emb = Embedding(fra_vocab_size, 256)(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
decoder_outputs, _, _= decoder_lstm(dec_masking, initial_state = encoder_states)

decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [24]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256)    1193472     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    1875456     input_2[0][0]                    
_______________________________________________________________________________________

In [25]:
model.compile(optimizer="rmsprop", loss='sparse_categorical_crossentropy', metrics=['acc'])

In [26]:
model.fit([encoder_input_train, decoder_input_train],decoder_target_train,
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 32, epochs = 50)

Epoch 1/50
938/938 [==============================] - 19s 21ms/step - loss: 1.5271 - acc: 0.7613 - val_loss: 1.2567 - val_acc: 0.8019
Epoch 2/50
938/938 [==============================] - 18s 19ms/step - loss: 1.1435 - acc: 0.8166 - val_loss: 1.0918 - val_acc: 0.8274
Epoch 3/50
938/938 [==============================] - 18s 19ms/step - loss: 1.0098 - acc: 0.8352 - val_loss: 0.9972 - val_acc: 0.8398
Epoch 4/50
938/938 [==============================] - 18s 19ms/step - loss: 0.9203 - acc: 0.8476 - val_loss: 0.9353 - val_acc: 0.8481
Epoch 5/50
938/938 [==============================] - 18s 19ms/step - loss: 0.8537 - acc: 0.8578 - val_loss: 0.8928 - val_acc: 0.8538
Epoch 6/50
938/938 [==============================] - 18s 19ms/step - loss: 0.7982 - acc: 0.8663 - val_loss: 0.8583 - val_acc: 0.8599
Epoch 7/50
938/938 [==============================] - 18s 19ms/step - loss: 0.7557 - acc: 0.8742 - val_loss: 0.8393 - val_acc: 0.8647
Epoch 8/50
938/938 [==============================] - 18s 19ms

## Test

### encoder

In [27]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 256)         1193472   
_________________________________________________________________
masking (Masking)            (None, None, 256)         0         
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 525312    
Total params: 1,718,784
Trainable params: 1,718,784
Non-trainable params: 0
_________________________________________________________________


### decoder

In [28]:
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = Embedding(fra_vocab_size, 256)(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = decoder_softmax_layer(decoder_outputs2)

In [29]:
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs2] + decoder_states2)
decoder_model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 256)    1875456     input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 256)]        0                                            
_______________________________________________________________________________________

In [30]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [36]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = fra2idx['<start>']
    
    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '<end>' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [37]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + idx2eng[i]+' '
    return temp

In [38]:
# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=fra2idx['<start>']) and i!=fra2idx['<end>']):
            temp = temp + idx2fra[i] + ' '
    return temp

In [48]:
for seq_index in [1,100,301,777,2222]:
    input_seq = encoder_input_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', seq2src(encoder_input_test[seq_index]))
    print('정답 문장:', seq2tar(decoder_input_test[seq_index]))
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1])

-----------------------------------
입력 문장: they re all alike . 
정답 문장: ils sont tous semblables . 
번역기가 번역한 문장:  elles sont tous tou
-----------------------------------
입력 문장: what do you see ? 
정답 문장: que voyez vous ? 
번역기가 번역한 문장:  que vous vous ? 
-----------------------------------
입력 문장: don t be so angry . 
정답 문장: ne te mets pas en col re . 
번역기가 번역한 문장:  ne ne pas pas ! 
-----------------------------------
입력 문장: she can handle it . 
정답 문장: elle sait y faire . 
번역기가 번역한 문장:  elle peut cela . cel
-----------------------------------
입력 문장: you re demented . 
정답 문장: vous tes fous . 
번역기가 번역한 문장:  vous tes tes . . vou


## Report

seq2seq를 이용하여 번역기(영어-프랑스어)를 만들어 봤습니다.  
단어 단위에서 구현하였습니다.  
<br>
결과는 다음과 같습니다.  
<br>
입력 문장: they re all alike .   
정답 문장: ils sont tous semblables .   
번역기가 번역한 문장:  elles sont tous tou  

입력 문장: what do you see ?   
정답 문장: que voyez vous ?   
번역기가 번역한 문장:  que vous vous ?   

입력 문장: don t be so angry .  
정답 문장: ne te mets pas en col re .  
번역기가 번역한 문장:  ne ne pas pas !  

입력 문장: she can handle it .  
정답 문장: elle sait y faire .  
번역기가 번역한 문장:  elle peut cela . cel  

입력 문장: you re demented .  
정답 문장: vous tes fous .  
번역기가 번역한 문장:  vous tes tes . . vou  
<br>
어느정도 유사하지만 결과가 완벽하지는 않습니다.